In [1]:
import datetime
import h5py
import numpy as np
import os
import pandas as pd
import sys
import time

sys.path.append('../src')
import localmodule


# Define constants.
args = ["unit01", "thrush"] #                            DISABLE
#args = sys.argv[1:]                                     ENABLE
unit_str = args[0]
odf_str = args[1]
data_dir = localmodule.get_data_dir()
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
oldbird_models_dir = os.path.join(models_dir, "oldbird")
unit_dir = os.path.join(oldbird_models_dir, unit_str)


# Define input and output folder.
in_predictions_name = "_".join(["predictions", "no-clip-suppressor"])
in_predictions_dir = os.path.join(unit_dir, in_predictions_name)
out_predictions_name = "_".join(["predictions", "clip-suppressor"])
out_predictions_dir = os.path.join(unit_dir, out_predictions_name)
os.makedirs(out_predictions_dir, exist_ok=True)


# Define suppressor count threshold and period.
oldbird_data_name = "_".join([dataset_name, "oldbird"])
oldbird_data_dir = os.path.join(data_dir, oldbird_data_name)
oldbird_data_path = os.path.join(oldbird_data_dir, unit_str + ".hdf5")
oldbird_hdf5 = h5py.File(oldbird_data_path, "r")
settings_key = "_".join([odf_str, "settings"])
settings = oldbird_hdf5[settings_key]
suppressor_count_threshold = settings["suppressor_count_threshold"].value
suppressor_period = settings["suppressor_period"].value

In [19]:
n_thresholds = 100
threshold_id = 0
# for threshold_id in range(n_thresholds):
threshold_str = "th-" + str(threshold_id).zfill(2)
in_prediction_name = "_".join(
    [dataset_name, "oldbird", odf_str,
     unit_str, threshold_str, "predictions.csv"])
in_prediction_path = os.path.join(in_predictions_dir, in_prediction_name)
df = pd.read_csv(in_prediction_path)
in_times = np.array(df["Time (s)"])

n_rows = len(in_times)
n_rows = 100
n = 0

In [26]:
n = 0
while n < n_rows:
    current_time = in_times[n]
    next_n = n + suppressor_count_threshold
    next_n = min(next_n, n_rows - 1)
    next_time = in_times[next_n]
    time_difference = next_time - current_time
    if time_difference < suppressor_period:
        print(n, time_difference, next_n - n)
        while time_difference < suppressor_period:
            next_n = next_n + 1
            next_time = in_times[next_n]
            time_difference = next_time - current_time
        n = next_n - 1
    # TODO append n
    n = n + 1

0 18.86 10
31 19.86 10
94 19.75 5


In [29]:
df[90:140]

,Dataset,Unit,ODF,Threshold ID,Upward threshold,Downward threshold,Time (s),Duration (s),Onset ODF,Offset ODF
90,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,258.80,0.40,1.162,1.001
91,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,278.85,0.18,1.282,0.832
92,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,282.14,0.40,1.177,0.916
93,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,286.15,0.18,1.219,0.849
94,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,286.76,0.40,1.148,0.974
95,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,289.02,0.40,1.151,0.996
96,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,296.39,0.18,1.177,0.833
97,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,303.49,0.18,1.142,0.842
98,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,305.42,0.40,1.163,1.017
99,BirdVox-70k,unit01,Old Bird (thrush),0,1.14,0.877,306.51,0.18,1.146,0.854
